In [30]:
using StochasticDiffEq
using DiffEqCallbacks
using DifferentialEquations
using Plots
using LinearAlgebra
using DelimitedFiles
using DataFrames
using CSV
using FileIO
using LinearAlgebra
using NPZ
using Random, Distributions
using SparseArrays
using LightGraphs
using BenchmarkTools


In [90]:
include("/home/callum/Julia/Modules/ASF_input.jl");
include("/home/callum/Julia/Modules/ASF_model.jl");

In [91]:
input_path = "/home/callum/Julia/ASF_Input/";

In [92]:
data_1 = ASF_Inputs.Model_Data(input_path);

-------------------------------------
Single Population!
The Only Population is Seeded With ASF!
-------------------------------------


In [39]:
nt = data_1.Parameters.Populations.cum_sum[end] #total number of groups and farms
nc = 5 #number of classes
eqs = 10#number of processes
n_ens = 100 #number of runs in ensemble

dc = sparse(zeros(nt*nc,nt*eqs))

#const _dc = zeros(nt*5,nt*10)

dc[0*nc*nt+1:nc*nt*eqs+nc:end] .= 1
dc[1*nc*nt+1:nc*nt*eqs+nc:end] .= -1
dc[2*nc*nt+1:nc*nt*eqs+nc:end] .= -1

dc[2*nc*nt+2:nc*nt*eqs+nc:end] .= 1
dc[3*nc*nt+2:nc*nt*eqs+nc:end] .= -1
dc[4*nc*nt+2:nc*nt*eqs+nc:end] .= -1

dc[4*nc*nt+3:nc*nt*eqs+nc:end] .= 1
dc[5*nc*nt+3:nc*nt*eqs+nc:end] .= -1
dc[6*nc*nt+3:nc*nt*eqs+nc:end] .= -1
dc[7*nc*nt+3:nc*nt*eqs+nc:end] .= -1

dc[7*nc*nt+4:nc*nt*eqs+nc:end] .= 1
dc[8*nc*nt+4:nc*nt*eqs+nc:end] .= -1

dc[5*nc*nt+5:nc*nt*eqs+nc:end] .= 1
dc[6*nc*nt+5:nc*nt*eqs+nc:end] .= 1
dc[9*nc*nt+5:nc*nt*eqs+nc:end] .= -1;

In [40]:
function regular_c(du,u,p,t,counts,mark)  
    mul!(du,dc,counts)
end


regular_c (generic function with 1 method)

In [96]:
rj = RegularJump(ASF_Model.frequency_rate,regular_c,eqs*nt)

jumps = JumpSet(rj)

prob = DiscreteProblem(data_1.U0,data_1.Time,data_1.Parameters)

jump_prob = JumpProblem(prob,Direct(),rj) #rng=rng)

#sol = solve(jump_prob,SimpleTauLeaping(), dt=.1)

ensemble_prob_m = EnsembleProblem(jump_prob)#,prob_func=prob_func)
output = solve(ensemble_prob_m, SimpleTauLeaping(), trajectories= n_ens,EnsembleThreads(),dt=.1);

In [97]:
ASF_Model.analyse_out(output,n_ens, data_1.Parameters.Populations)

------------------------------------------------
Run Stats:
1 Populations
100 Runs
------------------------------------------------
% of groups exposed to ASF in each population:
[97.62]
------------------------------------------------
% of groups that survive in each population:
[63.12]
------------------------------------------------
% of farms exposed to ASF in each population:
[NaN]
------------------------------------------------
% of farms that survive in each population:
[NaN]


In [106]:
rj = RegularJump(ASF_Model.density_rate,regular_c,eqs*nt)

jumps = JumpSet(rj)

prob = DiscreteProblem(data_1.U0,data_1.Time,data_1.Parameters)

jump_prob = JumpProblem(prob,Direct(),rj) #rng=rng)

#sol = solve(jump_prob,SimpleTauLeaping(), dt=.1)

ensemble_prob_m = EnsembleProblem(jump_prob)#,prob_func=prob_func)
output = solve(ensemble_prob_m, SimpleTauLeaping(), trajectories= n_ens,EnsembleThreads(),dt=.1);

In [107]:
ASF_Model.analyse_out(output,n_ens, data_1.Parameters.Populations)

------------------------------------------------
Run Stats:
1 Populations
100 Runs
------------------------------------------------
% of groups exposed to ASF in each population:
[92.85]
------------------------------------------------
% of groups that survive in each population:
[65.77]
------------------------------------------------
% of farms exposed to ASF in each population:
[NaN]
------------------------------------------------
% of farms that survive in each population:
[NaN]


In [ ]:
summ = EnsembleSummary(output)
pyplot() # Note that plotly does not support ribbon plots
plot(summ,fillalpha=0.5)

In [ ]:
output = hcat(sol.u...)
So = transpose(output[1:5:end,:])
Eo = transpose(output[2:5:end,:])
Io = transpose(output[3:5:end,:])
Ro = transpose(output[4:5:end,:])
Co = transpose(output[5:5:end,:])
